In [ ]:
from lec_utils import *
import lec22_util as util
from ipywidgets import FloatSlider, interact
from IPython.display import YouTubeVideo

<div class="alert alert-info" markdown="1">

#### Lecture 22

# Gradient Descent

### EECS 398-003: Practical Data Science, Fall 2024

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/fa24">github.com/practicaldsc/fa24</a></small>
    
</div>

<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    TeX: {
      extensions: ["color.js"],
      packages: {"[+]": ["color"]},
    }
  });
  </script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-AMS_HTML"></script>

### Announcements 📣

- The Portfolio Homework has been released! Read all about it [**here**](https://practicaldsc.org/portfolio). It has two due dates:
    - A checkpoint (worth 15 points / 100) is due on **Monday, November 25th** (no slip days!).
    - The full homework is due on **Saturday, December 7th** (no slip days!).
- Homework 10 will be out later this week.
- The [**Grade Report**](https://www.gradescope.com/courses/823979/assignments/5191081) now includes scores and slip days through Homework 8.

### Agenda

- Gradient descent intuition.
- When is gradient descent guaranteed to work?
- Gradient descent for multivariate functions.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Gradient descent intuition

---

### Minimizing arbitrary functions

- Assume $f(w)$ is some **differentiable** function.<br><small>For now, we'll assume $f$ takes in a single number, $w$, as input and returns a single number as its output.</small>

- When tasked with minimizing $f(w)$, our general strategy has been to:<br>
    1. Find $\frac{df}{dw}(w)$, the derivative of $f$.
    2. Find the input $w^*$ such that $\frac{df}{dw}(w^*) = 0$.


- However, there are cases where we can find $\frac{df}{dw}(w)$, but **it is either difficult or impossible to solve $\frac{df}{dw}(w^*) = 0$**.

$$f(w) = 5w^4 - w^3 - 5w^2 + 2w - 9$$

- Then what?

In [ ]:
util.draw_f()

### What does the derivative of a function tell us?

- **Goal**: Given a **differentiable** function $f(w)$, find the input $w^*$ that minimizes $f(w)$.

- What does $\frac{d}{dw} f(w)$ mean?

In [ ]:
from ipywidgets import interact
interact(util.show_tangent, w0=(-1.5, 1.5));

### Let's go hiking!

- Suppose you're at the top of a mountain 🏔️ and need to get **to the bottom**.

- Further, suppose it's really cloudy ☁️, meaning you can only see a few feet around you.

- **How** would you get to the bottom?


<center><img src="imgs/mountain.jpeg" width=500></center>

### Searching for the minimum

- Suppose we're given an initial _guess_ for a value of $w$ that minimizes $f(w)$.

<center>
    
<img src="imgs/positive-slope.png" width=500>
    
</center>

- If the <span style="color:red">**slope of the tangent line at $f(w)$**</span> is **positive** 📈:
    - Increasing $w$ **increases** $f$.
    - This means the minimum must be to the **left** of the point $(w, f(w))$.
    - Solution: **Decrease** $w$ ⬇️.

- The steeper the slope is, the further we must be from the minimum – so, the steeper the slope, the quicker we should decrease $w$!

### Searching for the minimum

- Suppose we're given an initial _guess_ for a value of $w$ that minimizes $f(w)$.

<center>
    
<img src="imgs/negative-slope.png" width=500>
    
</center>

- If the <span style="color:red">**slope of the tangent line at $f(w)$**</span> is **negative** 📉:
    - Increasing $w$ **decreases** $f$.
    - This means the minimum must be to the **right** of the point $(w, f(w))$.
    - Solution: **Increase** $w$ ⬆️.

- The steeper the slope is, the further we must be from the minimum – so, the steeper the slope, the quicker we should increase $w$!

### Intuition

- To minimize $f(w)$, start with an initial guess for the minimizing input, $w^{(0)}$.

- Where do we go next?
    - If $\frac{df}{dw}(w^{(0)}) > 0$, **decrease $w^{(0)}$**.
    - If $\frac{df}{dw}(w^{(0)}) < 0$, **increase $w^{(0)}$**.

- One way to accomplish this:

$$w^{(1)} = w^{(0)} - \frac{df}{dw}(w^{(0)})$$

- A consequence of the above **update rule**: the larger $\frac{df}{dw}$ is, the bigger a step we take!<br><small>This matches our intuition from the previous flew slides – the further we are from the minimum, the bigger of a step we should take!</small>

### Gradient descent

- To minimize a **differentiable** function $f$:
    1. Pick a positive number, $\alpha$. This number is called the **learning rate**, or **step size**.<br><small>Think of $\alpha$ as a hyperparameter of the minimization process.</small>
    2. Pick an **initial guess**, $w^{(0)}$.
    3. Then, repeatedly update your guess using the **update rule**:

$$w^{(t+1)} = w^{(t)} - \alpha \frac{df}{dw}(w^{(t)})$$

<br><br>

- Repeat this process until **convergence** – that is, when $w$ doesn't change much from iteration to iteration.

- This procedure is called **gradient descent**.

### What is gradient descent?

- Gradient descent is a numerical method for finding the input to a function $f$ that minimizes the function.

- It is called **gradient descent** because the gradient is the extension of the derivative to functions of multiple variables.

- A **numerical method** is a technique for approximating the solution to a mathematical problem, often by using the computer.

- Gradient descent is **widely used** in machine learning, to train models from linear regression to neural networks and transformers (includng ChatGPT)!<br><small>In machine learning, we use gradient descent to minimize empirical risk when we can't minimize it by hand, which is true in most, more sophisticated cases.</small>

### Implementing gradient descent

- In practice, we typically don't implement gradient descent ourselves – we rely on existing implementations of it. But, we'll implement it here ourselves to understand what's going on.

- Let's start with an initial guess $w^{(0)} = 0$ and a learning rate $\alpha = 0.01$.

$$w^{(t+1)} = w^{(t)} - \alpha \frac{df}{dw}(w^{(t)})$$

In [ ]:
...

- We see that pretty quickly, $w^{(t)}$ converges to $-0.727$!

### Visualizing $w^{(0)} = 0, \alpha = 0.01$

In [ ]:
util.minimizing_animation(w0=0, alpha=0.01)

### Visualizing $w^{(0)} = 1.1, \alpha = 0.01$

What if we start with a different initial guess?

In [ ]:
util.minimizing_animation(w0=1.1, alpha=0.01)

### Visualizing $w^{(0)} = 0, \alpha = 0.1$

What if we use a different learning rate?

In [ ]:
util.minimizing_animation(w0=0, alpha=0.1)

### Visualizing $w^{(0)} = 0, \alpha = 1$

Some learning rates are so large that the values of $w$ explode towards infinity! Watch what happens when we use a learning rate of 1:

In [ ]:
w = 0
for t in range(50):
    print(round(w, 4), round(util.f(w), 4))
    w = w - 1 * util.df(w)

### Gradient descent and empirical risk minimization

- While gradient descent can minimize other kinds of differentiable functions, its most common use case is in **minimizing empirical risk**.

- For example, consider:
    - The constant model, $H(x) = h$.
    - The dataset $-4, -2, 2, 4$.
    - The initial guess $h_0 = 4$ and the learning rate $\alpha = \frac{1}{4}$.


- **Exercise**: Find $h_1$ and $h_2$.

- See the annotated slides for the solution!

### Lingering questions

- When is gradient descent _guaranteed_ to converge to a global minimum? What kinds of functions work well with gradient descent?

- How do we choose a step size?

- How do we use gradient descent to minimize functions of multiple variables, e.g.:

$$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d w_j^2$$

- **Question**: Why **can't** we use gradient descent to find $\vec{w}_\text{LASSO}^*$?

$$R_\text{LASSO}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d |w_d|$$

## When is gradient descent guaranteed to work?

---

### What makes a function convex?

<div style="width: 100%;">
<div style="width: 50%; float: left"> 

<br>

<img src="imgs/convex.png">
<center>A <b>convex</b> function ✅.</center>
</div>

<div style="margin-left: 50%" markdown="1"> 

<br>

<img src="imgs/non-convex.png">
<center>A <b>non-convex</b> function ❌.</center>

</div/>
</div/>

### Intuitive definition of convexity

<div style="width: 100%;">
<div style="width: 50%; float: left"> 

<br>

<center><img src="imgs/convex.png" width=70%></center>
<center>A <b>convex</b> function ✅.</center>
</div>

<div style="margin-left: 50%" markdown="1"> 

<br>

<center><img src="imgs/non-convex.png" width=70%></center>
<center>A <b>non-convex</b> function ❌.</center>

</div/>
</div/>

- A function $f$ is **convex** if, for **every** $a, b$ in the domain of $f$, the line segment between:

  $$(a, f(a)) \text{ and } (b, f(b))$$

  does not go below the plot of $f$.

### Formal definition of convexity

<div style="width: 100%;">
<div style="width: 55%; float: left"> 

- A function $f: \mathbb{R} \rightarrow \mathbb{R}$ is **convex** if, for **every** $a, b$ in the domain of $f$, and for every $t \in [0, 1]$:

<br>

$$\boxed{(1 - t) f(a) + t f(b) \geq f((1-t)a + tb)}$$

<br><br><br>

- This is a formal way of restating the definition from the previous slide.


</div>

<div style="margin-left: 57%" markdown="1"> 

<br>

<center>

<img src="imgs/convex-definition.png" width=100%>

</center>

</div/>
</div/>

Here's an interactive version of the formal definition of convexity from the previous slide.

In [ ]:
interact(util.convexity_visual, a=(-20, 5, 0.1), b=(5, 20, 0.1), t=FloatSlider(min=0, max=1, step=0.01, value=0.5));

<div class="alert alert-success">
    
### Activity
    

Which of these functions are **not** convex?

- A. $f(x) = |x|$.
- B. $f(x) = e^x$.
- C. $f(x) = \sqrt{x-1}$.
- D. $f(x) = (x-3)^{24}$.
- E. More than one of the above are non-convex.
    
</div>

### Second derivative test for convexity

- If $f(t)$ is a function of a single variable and is **twice** differentiable, then $f(w)$ is convex **if and only if**:

$$\frac{d^2f}{dw^2}(w) \geq 0, \:\:\: \forall \: w$$

- Example: $f(x) = x^4$ is convex.

### Why does convexity matter?

- Convex functions are (relatively) easy to minimize with gradient descent.

- **Theorem**: If $f(w)$ is convex and differentiable, then gradient descent converges to a **global minimum** of $f$, as long as the step size is small enough.

- **Why?**
  - Gradient descent converges when the derivative is 0.
  - For convex functions, the derivative is 0 only at one place – the global minimum.
  - In other words, if $f$ is convex, gradient descent won't get "stuck" and terminate in places that aren't global minimums (local minimums, saddle points, etc.).

### Nonconvex functions and gradient descent

- We say a function is **nonconvex** if it does not meet the criteria for convexity.

- Nonconvex functions are (relatively) difficult to minimize.

- Gradient descent **might** still work, but it's not guaranteed to find a global minimum.
  - We saw this at the start of the lecture, when trying to minimize $f(w) = 5w^4 - w^3 - 5w^2 + 2w - 9$.

### Choosing a step size in practice

- In practice, choosing a step size involves a lot of trial-and-error.

- In this class, we've only touched on "constant" step sizes, i.e. where $\alpha$ is a constant.

$$w^{(t+1)} = w^{(t)} - \alpha \frac{df}{dw}(w^{(t)})$$

- **Remember**: $\alpha$ is the "step size", but the amount that our guess for $w$ changes is $\alpha \frac{df}{dw}(w^{(t)})$, not just $\alpha$.

- In future courses, you may learn about "decaying" step sizes, where the value of $\alpha$ decreases as the number of iterations increases.<br><small>Intuition: take much bigger steps at the start, and smaller steps as you progress, as you're likely getting closer to the minimum.</small>

## Gradient descent for empirical risk minimization

---

### Minimizing functions of multiple variables

- Consider the function:

$$f(x_1, x_2) = (x_1-2)^2 + 2x_1 - (x_2-3)^2$$

- It has two **partial derivatives**: $\frac{\partial f}{\partial x_1}$ and $\frac{\partial f}{\partial x_2}$.<br><small>See the annotated slides for what they are and how we find them.</small>

### The gradient vector

- If $f(\vec{x})$ is a function of multiple variables, then its **gradient**, $\nabla f (\vec{x})$, is a vector containing its partial derivatives.

- Example: 

$$f(\vec{x}) = (x_1-2)^2 + 2x_1 - (x_2-3)^2$$

$$\nabla f(\vec{x}) = \begin{bmatrix} 2(x_1 - 1) \\ -2(x_2 - 3)  \end{bmatrix}$$

- Example:

$$f(\vec{x}) = \vec{x}^T \vec{x}$$

$$\nabla f(\vec{x}) = 2 \vec{x}$$


<center>
<img src="imgs/surface.png" width=70%>
    
At any given point, there are many directions in which you can go "up", but there's only one "steepest direction up", and that's the direction of the gradient!
    
</center>

### Gradient descent for functions of multiple variables

- Example: 

$$f(x_1, x_2) = (x_1-2)^2 + 2x_1 - (x_2-3)^2$$

$$\nabla f (\vec{x}) = \begin{bmatrix} 2(x_1 - 1) \\ -2(x_2 - 3)  \end{bmatrix}$$

- The minimizer of $f$ is a vector, $\vec{x}^* = \begin{bmatrix} x_1^* \\ x_2^* \end{bmatrix}$.

- We start with an initial guess, $\vec{x}^{(0)}$, and step size $\alpha$, and update our guesses using:

$$\vec{x}^{(t+1)} = \vec{x}^{(t)} - \alpha \nabla f(\vec{x}^{(t)})$$

<div class="alert alert-success">
    
### Activity

$$f(x_1, x_2) = (x_1-2)^2 + 2x_1 - (x_2-3)^2$$
    
<br>

$$\nabla f (\vec{x}) = \begin{bmatrix} 2(x_1 - 1) \\ -2(x_2 - 3)  \end{bmatrix}$$
    
<br>

$$\vec{x}^{(t+1)} = \vec{x}^{(t)} - \alpha \nabla f(\vec{x}^{(t)})$$
    
<br>

Given an initial guess of $\vec{x}^{(0)} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}$ and a step size of $\alpha = \frac{1}{3}$, perform **two** iterations of gradient descent. What is $\vec{x}^{(2)}$?


### Example: Gradient descent for simple linear regression

- To find optimal model parameters for the model $H(x) = w_0 + w_1 x$ and squared loss, we minimized empirical risk:

$$R_\text{sq}(w_0, w_1) = R_\text{sq}(\vec{w}) = \frac{1}{n} \sum_{i = 1}^n ( y_i - (w_0 + w_1 x_i ))^2$$

- This is a function of multiple variables, and is differentiable, so it has a gradient!

$$\nabla R(\vec{w}) = \begin{bmatrix} \displaystyle -\frac{2}{n} \sum_{i = 1}^n (y_i - (w_0 + w_1 x_i)) \\ \displaystyle -\frac{2}{n} \sum_{i = 1}^n (y_i - (w_0 + w_1x_i))x_i  \end{bmatrix}$$

- **Key idea**: To find $\vec{w}^* = \begin{bmatrix} w_0^* \\ w_1^* \end{bmatrix}$, we _could_ use gradient descent!

- Why would we, when closed-form solutions exist?

### Gradient descent for simple linear regression, visualized

In [ ]:
YouTubeVideo('oMk6sP7hrbk')

### Gradient descent for simple linear regression, implemented

- Let's use gradient descent to fit a simple linear regression model to predict commute time in `'minutes'` from `'departure_hour'`.

In [ ]:
df = pd.read_csv('data/commute-times.csv')
df[['departure_hour', 'minutes']]
util.make_scatter(df)

In [ ]:
x = df['departure_hour']
y = df['minutes']

- First, let's remind ourselves what $w_0^*$ and $w_1^*$ are supposed to be.

In [ ]:
slope = np.corrcoef(x, y)[0, 1] * np.std(y) / np.std(x)
slope

In [ ]:
intercept = np.mean(y) - slope * np.mean(x)
intercept

### Implementing partial derivatives

$$R_\text{sq}(\vec{w}) = \frac{1}{n} \sum_{i = 1}^n ( y_i - (w_0 + w_1 x_i ))^2$$

$$\nabla R(\vec{w}) = \begin{bmatrix} \displaystyle -\frac{2}{n} \sum_{i = 1}^n (y_i - (w_0 + w_1 x_i)) \\ \displaystyle -\frac{2}{n} \sum_{i = 1}^n (y_i - (w_0 + w_1x_i))x_i  \end{bmatrix}$$

In [ ]:
def dR_w0(w0, w1):
    return -2 * np.mean(y - (w0 + w1 * x))
def dR_w1(w0, w1):
    return -2 * np.mean((y - (w0 + w1 * x)) * x)

### Implementing gradient descent

- The update rule we'll follow is:

$$\vec{w}^{(t+1)} = \vec{w}^{(t)} - \alpha \nabla R(\vec{w}^{(t)})$$

- We can treat this as two separate update equations:

$$w_0^{(t+1)} = w_0^{(t)} - \alpha \frac{\partial R}{\partial w_0} (\vec{w}^{(t)}) \\ w_1^{(t+1)} = w_1^{(t)} - \alpha \frac{\partial R}{\partial w_1} (\vec{w}^{(t)})$$

- Let's initialize $w_0^{(0)} = 100$ and $w_1^{(0)} = -50$, and choose the step size $\alpha = 0.01$.<br><small>The initial guesses were just parameters that we thought might be close.</small>

In [ ]:
# We'll store our guesses so far, so we can look at them later.
def gradient_descent_for_regression(w0_initial, w1_initial, alpha, threshold=0.0001):
    w0, w1 = w0_initial, w1_initial
    w0_history = [w0]
    w1_history = [w1]
    while True:
        w0 = w0 - alpha * dR_w0(w0, w1)
        w1 = w1 - alpha * dR_w1(w0, w1)
        w0_history.append(w0)
        w1_history.append(w1)
        if np.abs(w0_history[-1] - w0_history[-2]) <= threshold:
            break
    return w0_history, w1_history

In [ ]:
w0_history, w1_history = gradient_descent_for_regression(0, 0, 0.01)

In [ ]:
w0_history[-1]

In [ ]:
w1_history[-1]

- It seems that we converge at the right value! But how many iterations did it take? What could we do to speed it up?

In [ ]:
len(w0_history)